# 1. Pfad zur Struktur und Orientierung eintragen

In [1]:
CIF_Dateiname = 'F-43m_303K_6.05_Keen_Hull_CollCode80218.cif'
Orientierung = [1,1,1]

# 2. Run

In [2]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import re

import Dans_Diffraction as dif

print(dif.module_info())

Python version 3.12.0 (tags/v3.12.0:0fb18b0, Oct  2 2023, 13:03:39) [MSC v.1935 64 bit (AMD64)]
Dans_Diffraction version 3.1.3 (25/10/23)
     numpy version: 1.26.2
matplotlib version: 3.8.2
   tkinter version: 8.6
     scipy version: None


In [3]:
xtl = dif.Crystal(CIF_Dateiname)

print(xtl)


###########################################
F-43m_303K_6.05_Keen_Hull_CollCode80218
Formula: CuI
Magnetic: False
a =  6.052 A,  b =  6.052 A,  c =  6.052 A
A =    90.00,  B =    90.00,  G =    90.00
Volume: 221.68 A^3

Density:  5.707 g/cm
  n Label Atom  u       v       w        Occ  Uiso
  0    I1    I  0.0000  0.0000  0.0000   1.00 0.0165
  1    I1    I  0.0000  0.5000  0.5000   1.00 0.0165
  2    I1    I  0.5000  0.0000  0.5000   1.00 0.0165
  3    I1    I  0.5000  0.5000  0.0000   1.00 0.0165
  4   Cu1   Cu  0.7350  0.7350  0.2650   0.25 0.0177
  5   Cu1   Cu  0.7350  0.2650  0.7350   0.25 0.0177
  6   Cu1   Cu  0.2650  0.7350  0.7350   0.25 0.0177
  7   Cu1   Cu  0.2650  0.2650  0.2650   0.25 0.0177
  8   Cu1   Cu  0.7350  0.2350  0.7650   0.25 0.0177
  9   Cu1   Cu  0.7350  0.7650  0.2350   0.25 0.0177
 10   Cu1   Cu  0.2650  0.2350  0.2350   0.25 0.0177
 11   Cu1   Cu  0.2650  0.7650  0.7650   0.25 0.0177
 12   Cu1   Cu  0.2350  0.7350  0.7650   0.25 0.0177
 13   Cu1   Cu  0.2

In [4]:
xtl.Scatter.setup_scatter(wavelength_a = 1.54059295, min_theta = 0, specular = Orientierung)

Scatter(F-43m_303K_6.05_Keen_Hull_CollCode80218 with 2 atomic positions, 96 symmetries)
       Type: xray
     Energy: 8.047838264400514 keV
 Wavelength: 1.54059295 A
 ---Settings---
      Powder units: tth
    Isotropic ADPs: True
  Specular Direction (reflection): ( 1, 1, 1)
Parallel Direction (transmission): ( 0, 0, 1)
      theta offset: 0.0
         min theta: 0
         max theta: 180.0
      min twotheta: -180.0
      max twotheta: 180.0
 ---X-Ray Settings---
 Waasmaier scattering factor: False
 ---Magnetic Settings---
   Mag. scattering: True
  Mag. form factor: True
         Polarised: False
      Polarisation: sp
       Pol. vector: (0,1,0)
 ---Resonant Settings---
   Azimuthal angle: 0
    Azimuthal ref.: (1,0,0)
               flm: (0,1,0)
  use e1e1 approx.: True



In [5]:
xtl.Scatter.print_ref_reflections()

'Energy =  8.048 keV\nRadiation: xray\nSpecular Direction = (1,1,1)\n( h, k, l) TwoTheta    Theta  Intensity\n( 1, 1, 1)    25.47    12.74   41973.90\n( 2, 2, 2)    52.32    26.16    4293.11\n( 3, 1, 3)    67.39    11.69   15934.20\n( 3, 3, 1)    67.39    11.69   15934.20\n( 1, 3, 3)    67.39    11.69   15934.20\n( 2, 4, 2)    77.15    19.10   22421.44\n( 2, 2, 4)    77.15    19.10   22421.44\n( 4, 2, 2)    77.15    19.10   22421.44\n( 5, 1, 1)    82.81     2.46   11219.24\n( 1, 5, 1)    82.81     2.46   11219.24\n( 1, 1, 5)    82.81     2.46   11219.24\n( 3, 3, 3)    82.81    41.40   11471.73\n( 4, 4, 0)    92.11    10.79   15741.78\n( 4, 0, 4)    92.11    10.79   15741.78\n( 0, 4, 4)    92.11    10.79   15741.78\n( 5, 3, 1)    97.70    20.29    8253.22\n( 5, 3,-1)    97.70     1.94    8253.22\n( 3,-1, 5)    97.70     1.94    8253.22\n(-1, 5, 3)    97.70     1.94    8253.22\n( 1, 3, 5)    97.70    20.29    8253.22\n(-1, 3, 5)    97.70     1.94    8253.22\n( 5, 1, 3)    97.70    20.29 

In [6]:
columns = ['h', 'k', 'l', '2th', 'I']
zeilen = [string.replace(' ', '') for string in [s[1:9] + ',' + s[10:19] + ',' + s[28:] for s in xtl.Scatter.print_ref_reflections().split('\n')[4:-2]]]

data_list = [line.split(',') for line in zeilen]

df = pd.DataFrame(data_list, columns=columns)

df = df.apply(pd.to_numeric, errors='coerce')

df['Struktur'] = CIF_Dateiname
df['Indexsumme'] = df.apply(lambda row: row['h'] + row['k'] + row['l'], axis=1)

df = df.sort_values(by=['I', 'Indexsumme', 'h', 'k', 'l'], ascending=[False, False, True, True, True])

df['rel I'] = df['I']/df['I'].max()
df['hkl'] = df.apply(lambda row: [row['h'], row['k'], row['l']], axis=1)

df['rho'] = df.apply(lambda row: xtl.Cell.angle(row['hkl'], xtl.Scatter._scattering_specular_direction), axis=1)
df['omega'] = df['2th']/2 - df['rho']

df.head(20)

,h,k,l,2th,I,Struktur,Indexsumme,rel I,hkl,rho,omega
0,1,1,1,25.47,41973.90,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,3,1.000000,"[1, 1, 1]",0.000000,12.735000
6,2,2,4,77.15,22421.44,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,8,0.534176,"[2, 2, 4]",19.471221,19.103779
5,2,4,2,77.15,22421.44,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,8,0.534176,"[2, 4, 2]",19.471221,19.103779
7,4,2,2,77.15,22421.44,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,8,0.534176,"[4, 2, 2]",19.471221,19.103779
4,1,3,3,67.39,15934.20,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,7,0.379622,"[1, 3, 3]",22.001714,11.693286
2,3,1,3,67.39,15934.20,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,7,0.379622,"[3, 1, 3]",22.001714,11.693286
3,3,3,1,67.39,15934.20,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,7,0.379622,"[3, 3, 1]",22.001714,11.693286
14,0,4,4,92.11,15741.78,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,8,0.375037,"[0, 4, 4]",35.264390,10.790610
13,4,0,4,92.11,15741.78,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,8,0.375037,"[4, 0, 4]",35.264390,10.790610
12,4,4,0,92.11,15741.78,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,8,0.375037,"[4, 4, 0]",35.264390,10.790610


# 3. Liste der asymetrischen Refelxe, die von oben messbar wären.

In [7]:
sdf = df[(df['rho'] != 0) & (df['omega'] > 0) & (df['2th'] < 125) & (df['rel I'] > 0.001)]
udf = sdf.drop_duplicates(subset=['2th', 'I'])
head = ['hkl', '2th', 'Struktur', 'rel I', 'rho', 'omega']
udf[head]

,hkl,2th,Struktur,rel I,rho,omega
6,"[2, 2, 4]",77.15,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.534176,19.471221,19.103779
4,"[1, 3, 3]",67.39,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.379622,22.001714,11.693286
14,"[0, 4, 4]",92.11,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.375037,35.264390,10.790610
31,"[0, 2, 6]",107.21,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.270070,43.088723,10.516277
10,"[1, 1, 5]",82.81,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.267291,38.942441,2.462559
19,"[1, 3, 5]",97.70,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.196627,28.560825,20.289175
36,"[3, 3, 5]",113.15,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.140717,14.420068,42.154932
28,"[2, 4, 4]",99.58,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.057209,15.793169,33.996831
40,"[2, 2, 6]",115.19,F-43m_303K_6.05_Keen_Hull_CollCode80218.cif,0.047715,29.496208,28.098792
